In [ ]:
from libs.ltspice_helper import convert_ltspice_step_simulation_data_to_csv

dataset_filename = "simulations_ltspice/2_optimizing_components_dataset.csv"
datasets = [
    {"file": "simulations_ltspice/2_optimizing_components.txt"},
]

convert_ltspice_step_simulation_data_to_csv(
    datasets=datasets, output_filename=dataset_filename)


In [4]:
import pandas as pd
import numpy as np

dataset_filename = "simulations_ltspice/2_optimizing_components_dataset.csv"
df = pd.read_csv(dataset_filename, low_memory=True, on_bad_lines='warn')
df = df.apply(pd.to_numeric, errors='coerce')
df = df.replace(np.nan, 0)

df = df[['Fsw', 'L', 'Ro', 'V(eff)', 'V(pin)', 'V(pout)', 'V(pmos)', 'V(pdiode)', 'V(pcc)', 'V(pci)', 'V(pco)', 'V(pli)', 'V(plo)']]
df = df.groupby(['Fsw', 'L', 'Ro']).mean()

df.to_csv("simulations_ltspice/2_optimizing_components_dataset_mean.csv")
del df


b'Skipping line 24907436: expected 52 fields, saw 63\n'


In [5]:
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import cm
from scipy.interpolate import griddata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("simulations_ltspice/2_optimizing_components_dataset_mean.csv")
df

Fsw         L     Ro    V(eff)      V(pin)     V(pout)    V(pmos)  \
0    40000.0  0.000033    0.5  0.861997  202.700976  174.783362   6.324351   
1    40000.0  0.000033    1.0  0.906199  204.408468  185.265779   3.986514   
2    40000.0  0.000033    4.0  0.945903  202.357398  191.462812   2.432876   
3    40000.0  0.000033   20.0  0.957922  195.650644  187.270976   2.181147   
4    40000.0  0.000033   30.0  0.954800  193.729689  184.760351   2.157858   
5    40000.0  0.000033  100.0  0.937296  183.612568  171.869473   2.083894   
6   100000.0  0.000022    1.0  0.890281  204.242357  181.897251   5.813967   
7   100000.0  0.000022    4.0  0.931867  201.514107  187.829823   4.046517   
8   100000.0  0.000022   20.0  0.939324  195.470308  183.640786   4.627117   
9   100000.0  0.000022   30.0  0.935111  193.164486  180.646936   5.113976   
10  100000.0  0.000022  100.0  0.909887  183.252092  166.700049   8.173812   
11  100000.0  0.000033    0.5  0.843361  203.773391  171.921006   9.053478   
12  100000.0  0.000033    1.0  0.891823  203.455966  181.440895   5.791922   
13  100000.0  0.000033    4.0  0.931605  201.001908  187.301632   4.155224   
14  100000.0  0.000033   20.0  0.936049  195.179581  182.699118   5.161358   
15  100000.0  0.000033   30.0  0.931224  192.768011  179.484736   5.775868   
16  100000.0  0.000033  100.0  0.898162  182.102179  163.551901  10.271960   
17  200000.0  0.000022    0.5  0.811653  204.365956  165.942324  13.535361   
18  200000.0  0.000022    1.0  0.864544  201.522774  174.263200   8.790327   
19  200000.0  0.000022    4.0  0.906087  196.430009  178.003484   6.884037   
20  200000.0  0.000022   20.0  0.902119  185.412338  167.250962   9.617730   
21  200000.0  0.000022   30.0  0.895286  181.157601  162.164987  11.054405   
22  200000.0  0.000022  100.0  0.879575  165.778210  145.872847  19.332978   
23  200000.0  0.000033    0.5  0.812728  203.599858  165.491992  13.486710   
24  200000.0  0.000033    1.0  0.866191  200.654389  173.737993   8.768430   
25  200000.0  0.000033    4.0  0.905879  195.952423  177.528473   7.018971   
26  200000.0  0.000033   20.0  0.898559  184.471727  165.724632  10.124765   
27  200000.0  0.000033   30.0  0.875568  190.678745  166.539603  10.766473   

    V(pdiode)     V(pcc)    V(pci)     V(pco)    V(pli)    V(plo)  
0   15.317496   7.264767  1.787682  -4.558441  0.512480  4.125875  
1   11.039770   8.532971  2.236382  -5.652639  0.880544  2.098789  
2    5.514119  11.171953  3.421814  -8.550645  1.964879  1.441847  
3    2.345311  17.849734  4.897298 -12.359719  1.614473  1.310715  
4    1.931275  18.730609  4.883557 -11.598017  0.942026  1.322756  
5    1.344064  25.606974  5.485970 -14.656133  0.720654  1.528379  
6   12.395654   8.726514  2.413176  -5.202330  0.794712  1.743527  
7    6.843673  11.172037  2.693803  -7.382739  0.863322  0.515025  
8    3.633825  21.725945  3.809369 -15.563853  0.755873  0.147770  
9    3.109338  24.857670  3.872321 -17.802843  0.664220  0.106802  
10   3.266027  36.384960  3.866718 -28.406840  0.337994  0.102092  
11  16.836155   7.904866  2.229914  -4.240381  0.472335  3.344855  
12  12.523595   8.495500  2.351389  -4.977677  0.540157  1.512106  
13   7.077355  12.106544  2.768194  -7.811105  0.499260  0.119002  
14   4.279558  21.225166  3.535262 -14.950665  0.176168 -0.187867  
15   3.905778  24.219062  3.548988 -17.001615 -0.103714 -0.299845  
16   5.394434  33.815278  3.410503 -27.786590 -0.013047 -0.264397  
17  18.841335   8.169142  2.446439  -4.380034  0.701562  3.304761  
18  14.597511   9.038685  2.684535  -5.260036  0.726189  1.576541  
19   9.139429  12.751305  3.101684  -8.351255  0.573118  0.219297  
20   6.851993  23.391345  3.946266 -18.418558  0.514824 -0.146558  
21   6.850998  23.736660  3.510994 -19.512005  0.290803 -0.231065  
22   8.853384  36.690041  3.733356 -41.770258  0.311436 -0.278517  
23  18.878055   7.909321  2.385263  -4.247988  0.539359  3.233495  
24  14.677461   9.233007  2.755539  -5.400092 

In [100]:
rows_key = 'L'
cols_key = 'Fsw'

rows_val = df[rows_key].unique().tolist()
cols_val = df[cols_key].unique().tolist()

x_key = 'Ro'
y_key = 'V(eff)'

plt.figure(figsize=(15, 5))

for r in range(len(rows_val)):
    df_row = df.loc[df[rows_key] == rows_val[r]]
    for col_val in cols_val:
        df_col = df_row.loc[df_row[cols_key] == col_val]
        plt.plot(df_col[x_key], df_col[y_key], label=f'{cols_key}={col_val}, {rows_key}={rows_val[r]}')

plt.legend()
plt.xlabel(f'{x_key} [Ω]')
plt.ylabel('eff')
plt.tight_layout()
plt.show()